In [ ]:
!hostname

jupyter-kaborg15


In [2]:
import os
os.chdir("/home/kaborg15/FREja/")
#os.chdir("..")

In [5]:
import pandas as pd
import numpy as np



CLIP_FOLDER = "Pickled_Data/Embeddings_CLIP/"
fashion_clip_embeddings = np.load(f"{CLIP_FOLDER}/clip_model/image_vectors.npy", allow_pickle=True)
EMBEDDING_PATHS = [CLIP_FOLDER]
dir_contents = os.listdir(EMBEDDING_PATHS[0])
dir_contents.sort()
dir_contents

['.ipynb_checkpoints',
 'ViT_H_14_E2E_Config.pkl',
 'ViT_H_14_E2E_Config.pkl738',
 'ViT_H_14_E2E_final.pkl',
 'ViT_H_14_E2E_final.pkl739',
 'catalog.pkl',
 'clip_model',
 'fashion-clip.pkl',
 'pictures',
 'used_picture_ids.pkl']

In [7]:
import Image_Embedding_File_Handler
import torch

def load_fashion_clip_embeddings(clip_path):
    pictures_df = pd.read_pickle(f"{clip_path}/pictures_df.pkl")
    fashion_clip_embeddings = np.load(f"{clip_path}/image_vectors.npy", allow_pickle=True)
    pictures_df["embeddings"] = list(fashion_clip_embeddings)
    pictures_df["embeddings"] = pictures_df["embeddings"].apply(lambda x: torch.from_numpy(x))
    return pictures_df[["id", "owner", "embeddings"]]

PKL_LOAD_NAMES = [
 'ViT_H_14_E2E_final.pkl'
 ]

EMBEDDING_NAMES = [pkl_name.split(".")[0] for pkl_name in PKL_LOAD_NAMES]

embeddings_dict = {}
for embedding_df in PKL_LOAD_NAMES:
    for embedding_folder in EMBEDDING_PATHS:
        embeddings_path = embedding_folder + embedding_df
        if os.path.exists(embeddings_path):
            break
    else:
        raise ValueError(f"{embedding_df} not found in {EMBEDDING_PATHS}")
    #embeddings_dict[embedding_df.split(".")[0]] = pd.read_pickle(embeddings_path)
    embeddings_dict[embedding_df.split(".")[0]] = Image_Embedding_File_Handler.load_embeddings(embeddings_path)
    print(f"Loaded {embedding_df} from {embeddings_path}...")

EMBEDDING_NAMES += ["CLIP"]
embeddings_dict["CLIP"] = load_fashion_clip_embeddings(f"{EMBEDDING_PATHS[0]}/clip_model")

Loaded ViT_H_14_E2E_final.pkl from Pickled_Data/Embeddings_CLIP/ViT_H_14_E2E_final.pkl...


In [8]:
# Rename and merge loaded dataframes
dict_items = iter(embeddings_dict.items())
first_key, first_df = next(dict_items)
model_embeddings_df = first_df.copy()
if "file_exists" in model_embeddings_df.columns:
    model_embeddings_df.drop("file_exists", axis=1, inplace=True)
model_embeddings_df.rename({"embeddings":first_key}, axis=1, inplace=True)
for key, item_df in dict_items:
    item_df = pd.DataFrame(item_df[["id", "embeddings"]])
    item_df.rename({"embeddings": key}, axis=1, inplace=True)
    model_embeddings_df = pd.merge(model_embeddings_df, item_df, on="id", how="inner")

# Load outfit tags and filter unused categories
PICKLED_DATA_FOLDER = "Pickled_Data/"
PICKLED_OUTFITS_FILE = "active_outfits_df.pkl"
OUTFITS_DROP_COLUMNS = ['owner', 'name', 'brand', 'isPublic', 'isDeleted', 'meta.validFrom', 'meta.validTo', "Outfit_size"]

outfits_df = pd.read_pickle(PICKLED_DATA_FOLDER + PICKLED_OUTFITS_FILE)
outfits_df.drop(OUTFITS_DROP_COLUMNS, axis=1, inplace=True)
#IGNORE_CATEGORIES = ["Size", "Brand", "brand", "Fit"]
# "brand" is an invalid category name, actual brand names are in "Brand"
IGNORE_CATEGORIES = ["Size", "brand", "Fit", "Shoe Size"]
outfits_df["filtered_tags"] = outfits_df.apply(lambda row: [tag for tag, cat in zip(row["outfit_tags"], row["tag_categories"]) if cat not in IGNORE_CATEGORIES], axis=1)
outfits_df["filtered_tag_categories"] = outfits_df.apply(lambda row: [cat for cat in row["tag_categories"] if cat not in IGNORE_CATEGORIES], axis=1)
outfits_df["filtered_tags"].explode().value_counts()

filtered_tags
Everyday           6758
Spring             3712
Fall               3410
Polyester          3340
Summer             2999
                   ... 
Selected Femme        1
Benedicte Utzon       1
Rat & Boa             1
Prati                 1
GAP                   1
Name: count, Length: 585, dtype: int64

In [9]:

TAG_WEIGHT_DF_PATH = f"Pickled_Data/tag_entropy_df.pkl"
TO_DROP_COLUMNS = ["Shoe Size", "brand"]
tag_weights_df = pd.read_pickle(TAG_WEIGHT_DF_PATH)
tag_weights_df.loc["default_weight"] = 1
tag_weights_df.loc["inverse_normalized"] = 1 - tag_weights_df.loc["normalized"]
tag_weights_df.drop(TO_DROP_COLUMNS, axis=1, inplace=True)
tag_weights_df

,Brand,Material,Category,Color,Size,Occasion,No category,Sleeve,Embellishment,Neckline,Waist,Pattern,Fit,Length
entropy,2.150160,2.052015,1.293279,1.930027,1.175349,1.831700,1.933907,1.204265,0.602437,1.310849,1.152163,0.902646,1.094322,1.033667
normalized,0.335775,0.549009,0.412464,0.655482,0.445367,0.714127,0.778261,0.579129,0.289711,0.630385,0.592095,0.463868,0.679941,0.642253
lower,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
upper,6.403574,3.737670,3.135494,2.944439,2.639057,2.564949,2.484907,2.079442,2.079442,2.079442,1.945910,1.945910,1.609438,1.609438
default_weight,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
inverse_normalized,0.664225,0.450991,0.587536,0.344518,0.554633,0.285873,0.221739,0.420871,0.710289,0.369615,0.407905,0.536132,0.320059,0.357747


In [ ]:
tags_list = outfits_df["filtered_tags"].explode().unique()
tags_list

In [11]:
# to_dict_df = tag_weights_df.copy()
# to_dict_df.set_index("category", inplace=True)
# mean_weights_dict = to_dict_df["mean_weight"].to_dict()
# default_weights_dict = to_dict_df["default_weight"].to_dict()

#entropy_weights_dict = tag_weights_df.to_dict(orient="index")["entropy"]
#normalized_dict = tag_weights_df.to_dict(orient="index")["normalized"]

# Use normalized and inversed entropy weights as well as default weights
inverse_normalized_dict = tag_weights_df.to_dict(orient="index")["inverse_normalized"]
default_weights_dict = tag_weights_df.to_dict(orient="index")["default_weight"]
inverse_normalized_dict

{'Brand': 0.6642250638021232,
 'Material': 0.45099088486520245,
 'Category': 0.5875357573816247,
 'Color': 0.3445177815044965,
 'Size': 0.5546330831592883,
 'Occasion': 0.28587272619545157,
 'No category': 0.2217385295534735,
 'Sleeve': 0.4208708368339431,
 'Embellishment': 0.710288940274548,
 'Neckline': 0.36961496253511683,
 'Waist': 0.4079054911922093,
 'Pattern': 0.5361318428719093,
 'Fit': 0.3200592889226679,
 'Length': 0.3577465213862313}

In [ ]:
from sklearn import preprocessing
import numpy as np
import torch

def convert_to_vector(tags, tag_cats, tag_list, weights_dict):
    tag_vector = np.zeros(len(tag_list))
    for tag, cat in zip(tags, tag_cats):
        tag_index = np.where(tag_list == tag)[0][0]
        tag_vector[tag_index] = round(weights_dict[cat] * 1, 2)    
    return torch.Tensor(tag_vector.reshape(1, -1))

outfits_df["tag_vector"] = outfits_df.apply(lambda row: convert_to_vector(row["filtered_tags"], row["filtered_tag_categories"], tags_list, inverse_normalized_dict), axis=1)
outfits_df["tag_vector_binary"] = outfits_df.apply(lambda row: convert_to_vector(row["filtered_tags"], row["filtered_tag_categories"], tags_list, default_weights_dict), axis=1)
outfits_df["tag_vector_random"] = [torch.randint(0,2, size=(1, len(tags_list))).float() for _ in range(len(outfits_df))]
outfits_df["tag_vector_shuffled"] = np.random.permutation(outfits_df["tag_vector"].values)
outfits_df

In [13]:
outfit_merge = pd.DataFrame(outfits_df[["id", "tag_vector", "tag_vector_binary", "tag_vector_random", "tag_vector_shuffled"]])
outfit_merge.rename({"id":"owner"}, axis=1, inplace=True)
encodings_df = pd.merge(model_embeddings_df, outfit_merge, on="owner", how="inner")

# Generate random tensor embeddings for each outfit.
for column in EMBEDDING_NAMES:
    encodings_sample_tensor = torch.tensor(np.vstack(encodings_df[column].iloc[:2000].to_list()), dtype=torch.float32)
    abs_encodings_sample_tensor = torch.abs(encodings_sample_tensor)
    print(column, encodings_sample_tensor.shape, encodings_sample_tensor.max(), encodings_sample_tensor.min(), encodings_sample_tensor.mean(), abs_encodings_sample_tensor.mean())

arb_tensor_shape = encodings_df[EMBEDDING_NAMES[0]].iloc[0].shape
generated_tensor = torch.rand(arb_tensor_shape)
print(f"Generating random embeddings with same shape as {EMBEDDING_NAMES[0]}({arb_tensor_shape})")

encodings_df["random"] = [torch.rand(arb_tensor_shape) for _ in range(len(encodings_df))]
encodings_df["random_shuffle"] = np.random.permutation(encodings_df[EMBEDDING_NAMES[0]].values)
encodings_df

ViT_H_14_E2E_final torch.Size([2000, 1280]) tensor(5.5742) tensor(-5.1641) tensor(-0.0408) tensor(0.8832)
CLIP torch.Size([2000, 512]) tensor(0.4263) tensor(-0.4259) tensor(-0.0003) tensor(0.0306)
Generating random embeddings with same shape as ViT_H_14_E2E_final(torch.Size([1280]))


,id,owner,ViT_H_14_E2E_final,CLIP,tag_vector,tag_vector_binary,tag_vector_random,tag_vector_shuffled,random,random_shuffle
0,picture.00058abb53434872ae9bb4270ae21f8e,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-1.3496, dtype=torch.float16), tensor(...","[-0.003936153, 0.12214454, -0.049501706, 0.022...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.4443), tensor(0.6602), tensor(0.8125...","[tensor(0.1771, dtype=torch.float16), tensor(0..."
1,picture.2911fa95ced4417d8cfced005367bf5b,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-0.5562, dtype=torch.float16), tensor(...","[-0.016397828, 0.08128633, -0.048218723, 0.004...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.3505), tensor(0.8759), tensor(0.8377...","[tensor(1.1299, dtype=torch.float16), tensor(1..."
2,picture.3be163f353de4bceaffdebbd22429a80,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-1.9121, dtype=torch.float16), tensor(...","[-0.025882766, 0.111839496, -0.059308674, -0.0...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.0594), tensor(0.1453), tensor(0.8640...","[tensor(0.5459, dtype=torch.float16), tensor(1..."
3,picture.5ba8cf9fa702483ab6db2a5e39ea24ff,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-2.2637, dtype=torch.float16), tensor(...","[0.02845695, 0.16913266, -0.011954581, 0.02296...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.0616), tensor(0.3098), tensor(0.8718...","[tensor(-1.2803, dtype=torch.float16), tensor(..."
4,picture.d55b9888fe31446cafd45c64be9b6702,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-2.3145, dtype=torch.float16), tensor(...","[-0.003647874, 0.13009632, -0.030201511, 0.027...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.3096), tensor(0.4402), tensor(0.6080...","[tensor(0.4290, dtype=torch.float16), tensor(1..."
...,...,...,...,...,...,...,...,...,...,...
26840,picture.fe22c4f30f774d5e8ed32cfc69dbda65,outfit.e17a0df1f70743e590827d7df93bb7f4,"[tensor(-0.8086, dtype=torch.float16), tensor(...","[-0.01972102, 0.0317783, 0.024466757, 0.033616...","[[tensor(0.4500), tensor(0.2200), tensor(0.290...","[[tensor(1.), tensor(1.), tensor(1.), tensor(1...","[[tensor(0.), tensor(1.), tensor(0.), tensor(0...","[[tensor(0.4500), tensor(0.2200), tensor(0.290...","[tensor(0.8060), tensor(0.9684), tensor(0.3569...","[tensor(0.4407, dtype=torch.float16), tensor(-..."
26841,picture.fe91d48a96c24600adfbbe27a1eb0946,outfit.ff9720b3fa3f42568a25aac897b66e6e,"[tensor(-1.7637, dtype=torch.float16), tensor(...","[-0.03119102, -0.045980014, -0.0061184377, 0.0...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(1.), tensor(1.), tensor(1.), tensor(1...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[tensor(0.4282), tensor(0.2770), tensor(0.1706...","[tensor(-0.1819, dtype=torch.float16), tensor(..."
26842,picture.fec9c77820734d8d866979323682eced,outfit.e2bde0d9e79f4e729b54482af02f0caf,"[tensor(-0.8696, dtype=torch.float16), tensor(...","[-0.011584576, 0.06986244, 0.07413015, 0.01163...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(1.), tensor(1.), tensor(0.)

In [14]:
from torch.nn import functional as F
from torchmetrics.classification import BinaryJaccardIndex, JaccardIndex
EXAMPLE_TARGET_INDEX = 0
target_vector = encodings_df.iloc[EXAMPLE_TARGET_INDEX]["tag_vector"]
test_vector = encodings_df.iloc[201]["tag_vector"]
jaccard_index = BinaryJaccardIndex()
F.cosine_similarity(target_vector, test_vector)

tensor([0.0891])

In [15]:
from torch.nn import functional as F
from torchmetrics.classification import BinaryJaccardIndex
VECTOR_DISTANCE_POINTS = "tag_vector"
target_vector = encodings_df.iloc[EXAMPLE_TARGET_INDEX][VECTOR_DISTANCE_POINTS]
jaccard_index = BinaryJaccardIndex()
#encodings_df["vector_distance"] = encodings_df["tag_vector"].apply(lambda x: jaccard_index(x, target_vector).item())
encodings_df["vector_distance"] = encodings_df[VECTOR_DISTANCE_POINTS].apply(lambda x: F.cosine_similarity(x, target_vector).item())
#encodings_df["vector_distance"] = encodings_df["vector_distance"].apply(lambda x: 1 - x)
display(encodings_df["vector_distance"].describe())
encodings_df.head()

count    26845.000000
mean         0.208527
std          0.143576
min          0.000000
25%          0.091813
50%          0.196989
75%          0.311739
max          1.000000
Name: vector_distance, dtype: float64

,id,owner,ViT_H_14_E2E_final,CLIP,tag_vector,tag_vector_binary,tag_vector_random,tag_vector_shuffled,random,random_shuffle,vector_distance
0,picture.00058abb53434872ae9bb4270ae21f8e,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-1.3496, dtype=torch.float16), tensor(...","[-0.003936153, 0.12214454, -0.049501706, 0.022...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.4443), tensor(0.6602), tensor(0.8125...","[tensor(0.1771, dtype=torch.float16), tensor(0...",1.0
1,picture.2911fa95ced4417d8cfced005367bf5b,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-0.5562, dtype=torch.float16), tensor(...","[-0.016397828, 0.08128633, -0.048218723, 0.004...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.3505), tensor(0.8759), tensor(0.8377...","[tensor(1.1299, dtype=torch.float16), tensor(1...",1.0
2,picture.3be163f353de4bceaffdebbd22429a80,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-1.9121, dtype=torch.float16), tensor(...","[-0.025882766, 0.111839496, -0.059308674, -0.0...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.0594), tensor(0.1453), tensor(0.8640...","[tensor(0.5459, dtype=torch.float16), tensor(1...",1.0
3,picture.5ba8cf9fa702483ab6db2a5e39ea24ff,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-2.2637, dtype=torch.float16), tensor(...","[0.02845695, 0.16913266, -0.011954581, 0.02296...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.0616), tensor(0.3098), tensor(0.8718...","[tensor(-1.2803, dtype=torch.float16), tensor(...",1.0
4,picture.d55b9888fe31446cafd45c64be9b6702,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(-2.3145, dtype=torch.float16), tensor(...","[-0.003647874, 0.13009632, -0.030201511, 0.027...","[[tensor(0.), tensor(0.), tensor(0.2900), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(0.4500), tensor(0.), tensor(0.), tens...","[tensor(0.3096), tensor(0.4402), tensor(0.6080...","[tensor(0.4290, dtype=torch.float16), tensor(1...",1.0


In [17]:
#encodings_df["CLIP"] = encodings_df["CLIP"].apply(lambda x: torch.from_numpy(x))

In [20]:
EMBEDDING_NAMES

['ViT_H_14_E2E_final', 'CLIP']

In [23]:
import copy

import wandb
LOG_WANDB = False
if LOG_WANDB:
    wandb.init(project="Image_similarity_tag_correlations", entity="kaborg15")

    WANDB_RUN_NAME = "tag_vector_entropy_1000"
    wandb.run.name = WANDB_RUN_NAME
from tqdm import tqdm

def distance_evaluator_altered(embedding_1, embedding_2):
    return F.cosine_similarity(embedding_1.float(), embedding_2.float(), dim=0).item()
    #return torch.dist(embedding_1, embedding_2, 2).item()

DISTANCE_COLUMN_EXSTENSION = "_distance0"

print("Evaluating distances using hard-coded column subset")

DISTANCE_COLUMNS_RUN = [
 ('CLIP', 1, 1),
 ('ViT_H_14_E2E_final', 1, 1)
]

TAG_VECTOR_COLUMNS = ["tag_vector", "tag_vector_binary", "tag_vector_random",	"tag_vector_shuffled"]
MULTIPLICATION_COLUMNS = copy.copy(DISTANCE_COLUMNS_RUN)
MULTIPLICATION_EXTENSION = "_sigmoid"
COLUMN_MULTIPLICATION_FACTOR = 2
VECTOR_DISTANCE_EXTENSION = "_distance"

wandb.config = {
    "distance_columns": DISTANCE_COLUMNS_RUN,
    "vector_columns": TAG_VECTOR_COLUMNS
}

jaccard_index = BinaryJaccardIndex()
sigmoid = torch.nn.Sigmoid()

average_encodings_df = encodings_df.copy()

model_correlations = {column[0]:np.empty(shape=(0,len(TAG_VECTOR_COLUMNS))) for column in DISTANCE_COLUMNS_RUN}
for i, row in tqdm(enumerate(average_encodings_df.sample(1000, random_state=42).index)):
    for tag_column in TAG_VECTOR_COLUMNS:
        target_vector = average_encodings_df.loc[row][tag_column]
        average_encodings_df[tag_column + VECTOR_DISTANCE_EXTENSION] = average_encodings_df[tag_column].apply(lambda x: F.cosine_similarity(x, target_vector)[0].item())
    
    for evaluation_column in DISTANCE_COLUMNS_RUN:
        if type(evaluation_column) == tuple:
            evaluation_column = evaluation_column[0]
        target_embeddings = average_encodings_df.loc[row][evaluation_column]
        average_encodings_df[evaluation_column + DISTANCE_COLUMN_EXSTENSION] = average_encodings_df[evaluation_column].apply(lambda x: distance_evaluator_altered(target_embeddings, x))
        average_encodings_df[evaluation_column + DISTANCE_COLUMN_EXSTENSION] = average_encodings_df[evaluation_column + DISTANCE_COLUMN_EXSTENSION].apply(lambda x: np.round(x, 4))
    
    closest_df = pd.DataFrame(average_encodings_df.groupby("owner").head(1))[["owner"] + [tag_column + VECTOR_DISTANCE_EXTENSION for tag_column in TAG_VECTOR_COLUMNS]]
    for resnet_column in DISTANCE_COLUMNS_RUN:
        if type(resnet_column) == tuple:
            resnet_column = resnet_column[0]
        column_name = resnet_column + DISTANCE_COLUMN_EXSTENSION
        tmp_df = pd.DataFrame(average_encodings_df.groupby("owner").head(1)).sort_values(by=column_name, ascending=True)[["owner", column_name]]
        closest_df = pd.merge(closest_df, tmp_df, on="owner", how="inner")
        tag_correlations = []
        for tag_column in TAG_VECTOR_COLUMNS:
            #tag_correlations.append(closest_df[tag_column + VECTOR_DISTANCE_EXTENSION].corr(closest_df[column_name]) ** 2)
            tag_correlations.append(closest_df[tag_column + VECTOR_DISTANCE_EXTENSION].corr(closest_df[column_name]))
        model_correlations[resnet_column] = np.vstack([model_correlations[resnet_column], np.array(tag_correlations)])

    if LOG_WANDB:
        wandb_dict = {}
        for column, correlations in model_correlations.items():
            wandb_dict = {**wandb_dict, **{f"{column}_{tag_column}":correlations[i,j] for j, tag_column in enumerate(TAG_VECTOR_COLUMNS)}}
        wandb.log(wandb_dict)

#for column, correlations in model_correlations.items():


for column in DISTANCE_COLUMNS_RUN:
    if type(column) == tuple:
        column, acc_1, acc_5 = column
    else:
        acc_1, acc_5 = -1, -1
    correlations = model_correlations[column]
    combined_accuracy = ((acc_1 + acc_5) / 2) / 100
    correlation_string, combined_accuracy_string = "", ""
    for j, tag_column in enumerate(TAG_VECTOR_COLUMNS):
        num_nan = np.isnan(correlations[:,j]).sum()
        if num_nan > 0:
            print(f"Warning: {num_nan} NaN values found in {column}")
        column_correlations = correlations[:,j][~np.isnan(correlations[:,j])]
        correlation_score = np.array(column_correlations).mean()
        correlation_string += f"({tag_column}: {round(correlation_score, 3)})"
        combined_accuracy_string += f"({tag_column}: {round(correlation_score / combined_accuracy, 3)})"
    print(f"{correlation_string} {column}: Average correlation between {column} and tags")
    #print(f"Acc score: {combined_accuracy_string} {column}: Accuracy correlation score {column} and tags, with an accuracy of {round(combined_accuracy, 3)} and a correlation of {round(correlation_score, 5)}")

Evaluating distances using hard-coded column subset


363it [36:57,  6.18s/it]

In [ ]:
correlation_string